# Signaling Adversity

(_A chapter name after my heart._)

## Nothing

A very common datatype in Haskell because it lets us return a default `Nothing` value when we don’t have any sensible values to return for our intended type `a`. Below is a function that is correct and explicit about the possibility of not getting a result when the `Integer` provided is odd.

In [14]:
ifEvenAdd2 :: Integer -> Maybe Integer
ifEvenAdd2 n =
    if even n
    then Just (n + 2)
    else Nothing

--

ifEvenAdd2 12
ifEvenAdd2 13

Just 14

Nothing

## Smart Constructors

By making use of `Maybe`.

In [15]:
type Name = String
type Age = Integer

data Person =
    Person Name Age
    deriving Show
    
mkPerson :: Name -> Age -> Maybe Person
mkPerson name age
    | name /= "" && age >= 0 =
        Just $ Person name age
    | otherwise = Nothing

--

mkPerson "John Browning" 160
mkPerson "" 160
mkPerson "blah" 0
mkPerson "blah" (-9001)

Just (Person "John Browning" 160)

Nothing

Just (Person "blah" 0)

Nothing

## Either

A way to know why our inputs were incorrect, if they were incorrect. It is conventional to use `Left` as our invalid or error constructor in Haskell.

data PersonInvalid =
    NameEmpty
    | AgeTooLow
    deriving (Eq, Show)
    
mkPerson :: Name -> Age -> Either PersonInvalid Person
mkPerson name age
    | name /= "" && age >= 0 = Right $ Person name age
    | name == ""             = Left NameEmpty
    | otherwise              = Left AgeTooLow

--

mkPerson "Djali" 5
mkPerson "" 10
mkPerson "Djali" (-1)
mkPerson "" (-1)

All of this to have `mkPerson` return a list of validation errors.

In [17]:
type ValidatePerson a = Either [PersonInvalid] a

ageOkay :: Age -> Either [PersonInvalid] Age
ageOkay age = case age >= 0 of
    True -> Right age
    False -> Left [AgeTooLow]

nameOkay :: Name -> Either [PersonInvalid] Name
nameOkay name = case name /= "" of
    True -> Right name
    False -> Left [NameEmpty]

mkPerson :: Name -> Age -> ValidatePerson Person
mkPerson name age =
    mkPerson' (nameOkay name) (ageOkay age)

mkPerson' :: ValidatePerson Name -> ValidatePerson Age -> ValidatePerson Person
mkPerson' (Right nameOk) (Right ageOk) =
    Right (Person nameOk ageOk)
mkPerson' (Left badName) (Left badAge) =
    Left (badName ++ badAge)
mkPerson' (Left badName) _ = Left badName
mkPerson' _ (Left badAge) = Left badAge

--

mkPerson "" (-1)

Left [NameEmpty,AgeTooLow]

## Kinds and Higher-kinded Types

_Type constructors_ (that is, higher-kinded types) are types that take more types as arguments. _Type constants_ refer to types that take no arguments and are already types.

The `Maybe` and `Either` datatypes we’ve just reviewed also have _type constructors_ rather than _type constants_.

```haskell
Prelude> :k Maybe
Maybe :: * -> *
Prelude> :k Maybe Int
Maybe Int :: *

Prelude> :k Either
Either :: * -> * -> *
Prelude> :k Either Int
Either Int :: * -> *
Prelude> :k Either Int String
Either Int String :: *
```

## Lifted and Unlifted Types

`kind *` is the kind of all standard lifted types, while types that have the `kind #` are unlifted. A lifted type is any that can be inhabited by _bottom_. Unlifted types are any type which cannot be inhabited by _bottom_.

Goal is to give us a way to understand when type errors are caused by things not being fully applied? Cool story.

```haskell
Prelude> :k Maybe
Maybe :: * -> *
Prelude> :k Maybe Int
Maybe Int :: *
Prelude> :k Maybe Bool
Maybe Bool :: *

Prelude> :k Maybe Maybe

Expecting one more argument to ‘Maybe’
The first argument of ‘Maybe’ should have kind ‘*’,
    but ‘Maybe’ has kind ‘* -> *’
In a type in a GHCi command: Maybe Maybe

Prelude> :k Maybe Char
Maybe Char :: *
Prelude> :k Maybe (Maybe Char)
Maybe (Maybe Char) :: *
```

## Data Constructors are Functions

Some examples of how data constructros can act like functions?

```haskell
Prelude> data Trivial = Trivial deriving Show
Prelude> Trivial 1

Couldn't match expected type ‘Integer -> t’
    with actual type ‘Trivial’
    
Prelude> data UnaryC = UnaryC Int deriving Show
Prelude> :t UnaryC UnaryC :: Int -> UnaryC
Prelude> UnaryC 10 UnaryC 10
Prelude> :t UnaryC 10 UnaryC 10 :: UnaryC

Prelude> data Unary a = Unary a deriving Show
Prelude> :t Unary Unary :: a -> Unary a
Prelude> :t Unary 10 Unary 10 :: Num a => Unary a
Prelude> :t Unary "blah" Unary "blah" :: Unary [Char]
```

# Exercises

1) Given

```haskell
id :: a -> a
```

What is the kind of `a`?

2) Given

```haskell
r :: a -> f a
```

What are the kinds of `a` and `f`?

## String Processing

1) Write a recursive function named `replaceThe` which takes a text/string, breaks it into words and replaces each instance of “the” with “a”. It’s intended only to replace exactly the word “the”. `notThe` is a suggested helper function for accomplishing this.

In [52]:
import Data.Char
import Data.Maybe

notThe :: String -> Maybe String
notThe "the" = Nothing
notThe word  = Just word

--

notThe "the"
notThe "blahtheblah"
notThe "woot"

--

replaceThe :: String -> String
replaceThe =
    unwords . map (fromMaybe "a" . notThe) . words

--

replaceThe "the cow loves us"

Nothing

Just "blahtheblah"

Just "woot"

"a cow loves us"

2) Write a recursive function that takes a text/string, breaks it into words, and counts the number of instances of ”the” followed by a vowel-initial word.

In [83]:
vowels :: [Char]
vowels = "aeiou"

beforeVowelInitial :: String -> Maybe String
beforeVowelInitial body@('t':'h':'e':' ':x:xs)
    | elem x vowels = Nothing
    | otherwise     = Just body
beforeVowelInitial xs = Just xs

countTheBeforeVowel :: String -> Integer
countTheBeforeVowel "" = 0
countTheBeforeVowel body =
    case beforeVowelInitial body of
        Nothing     -> 1 + countTheBeforeVowel (drop 4 body)
        Just (x:xs) -> 0 + countTheBeforeVowel xs

--

countTheBeforeVowel "the cow"
countTheBeforeVowel "the evil cow"
countTheBeforeVowel "the evil cow jumped over the ill moon"

0

1

2

3) Return the number of letters that are vowels in a word.

__Hint__: it’s helpful to break this into steps. Add any helper functions necessary to achieve your objectives.

- Test for vowelhood  
- Return the vowels of a string  
- Count the number of elements returned

In [95]:
countVowels :: String -> Integer
countVowels = toInteger . length . filter (`elem` vowels)

--

countVowels "the cow"
countVowels "Mikolajczak"

2

4

## Validate the Word

Use the `Maybe` type to write a function that counts the number of vowels in a string and the number of consonants. If the number of vowels exceeds the number of consonants, the function returns `Nothing`. In many human languages, vowels rarely exceed the number of consonants so when they do, it may indicate the input isn’t a word (that is, a valid input to your dataset):

In [111]:
newtype Word' =
    Word' String deriving (Eq, Show)

countConsonants :: String -> Integer
countConsonants = toInteger . length . filter (not . (`elem` vowels))

mkWord :: String -> Maybe Word'
mkWord xs
    | vs > cs   = Nothing
    | otherwise = Just (Word' xs)
    where vs = countVowels xs
          cs = countConsonants xs
          
--

mkWord "hello there"
mkWord "helloooooooo there"

Just (Word' "hello there")

Nothing

## It’s Only Natural

You’ll be presented with a datatype to represent the natural numbers. The only values representable with the naturals are whole numbers from zero to infinity. Your task will be to implement functions to convert `Naturals` to `Integers` and `Integers` to `Naturals`. The conversion from `Naturals` to `Integers` won’t return `Maybe` because `Integer` is a strict superset of `Natural`. Any `Natural` can be represented by an `Integer`, but the same is not true of any `Integer`. Negative numbers are not valid natural numbers.

In [115]:
data Nat =
    Zero
    | Succ Nat
    deriving (Eq, Show)

natToInteger :: Nat -> Integer
natToInteger Zero = 0
natToInteger (Succ x) = 1 + natToInteger x

--

natToInteger Zero
natToInteger (Succ Zero)
natToInteger (Succ (Succ Zero))

--

integerToNat :: Integer -> Maybe Nat
integerToNat 0 = Just Zero
integerToNat n
    | n < 0     = Nothing
    | otherwise = Just (Succ x)
    where (Just x) = integerToNat $ (n - 1)

--

integerToNat 0
integerToNat 1
integerToNat 2
integerToNat (-1)

0

1

2

Just Zero

Just (Succ Zero)

Just (Succ (Succ Zero))

Nothing

## Small Library for Maybe

Write the following functions. This may take some time. `:'(`

1) Simple boolean checks for `Maybe` values.

In [136]:
isJust' :: Maybe a -> Bool
isJust' Nothing  = False
isJust' _        = True

isNothing' :: Maybe a -> Bool
isNothing' Nothing = True
isNothing' _       = False

--

isJust' (Just 1)
isJust' Nothing

isNothing' (Just 1)
isNothing' Nothing

True

False

False

True

2) The following is the `Maybe` catamorphism. You can turn a `Maybe` value into anything else with this.

In [124]:
import Data.Maybe

mayybee :: b -> (a -> b) -> Maybe a -> b
mayybee d f x
    | isJust x  = f (fromJust x)
    | otherwise = d

--

mayybee 0 (+1) Nothing
mayybee 0 (+1) (Just 1)

0

2

3) In case you just want to provide a fallback value.

In [128]:
fromMaybe' :: a -> Maybe a -> a
fromMaybe' d x =
    case x of
        Nothing -> d
        Just v  -> v

--

fromMaybe' 0 Nothing
fromMaybe' 0 (Just 1)

0

1

4) Converting between `List` and `Maybe`.

In [135]:
listToMaybe' :: [a] -> Maybe a
listToMaybe' []     = Nothing
listToMaybe' (x:xs) = Just x

maybeToList' :: Maybe a -> [a]
maybeToList' Nothing  = []
maybeToList' (Just v) = [v]

--

listToMaybe' [1, 2, 3]
listToMaybe' []

maybeToList' (Just 1)
maybeToList' Nothing

Just 1

Nothing

[1]

[]

5) For when we want to drop the `Nothing` values from our list.

In [147]:
catMaybes' :: [Maybe a] -> [a]
catMaybes' [] = []
catMaybes' (x:xs) =
    case x of
        Nothing -> catMaybes' xs
        Just v  -> v : catMaybes' xs

--

catMaybes' [Just 1, Nothing, Just 2]
catMaybes' (replicate 3 Nothing)

[1,2]

[]

6) You’ll see this called “sequence” later.

In [160]:
flipMaybe :: [Maybe a] -> Maybe [a]
flipMaybe [] = Nothing
flipMaybe x =
    case n of
        0 -> Just $ catMaybes' x
        _ -> Nothing
    where n = length $ filter isNothing' x

--

flipMaybe [Just 1, Just 2, Just 3]
flipMaybe [Just 1, Nothing, Just 3]

Just [1,2,3]

Nothing